# 1.0 Fix data

AIMS 

En este archivo se realiza la limpieza de los archivos y el ETL

In [1]:
import pandas as pd
from datetime import datetime
from mypackage import dir, dataquality

In [2]:
%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [3]:
modality =  modality = 'p'
project = 'australia'
data = dir.make_dir_line(modality, project) 
raw = data('raw')

In [4]:
# # Using Value Counts
# discrete_vars_counts = [col for col in df_clientes.columns if df_clientes[col].nunique() < len(df_clientes) / 20]  # Assuming a threshold here

# # Print or use the identified discrete variables
# print("Discrete variables by value counts:", discrete_vars_counts)

def discrete_values(dataframe, threshold=10):
    # Using Value Counts
    discrete_vars_counts = [col for col in dataframe.columns if dataframe[col].nunique() < threshold]  # Assuming a threshold here

    # Print or use the identified discrete variables
    print("Discrete variables by value counts:", discrete_vars_counts)

def non_sex(dataframe, variable):
    # Label sex or gender
    dataframe[variable] = dataframe[variable].replace(['F', 'f', 'Femenino', 'femenino'], 0)
    dataframe[variable] = dataframe[variable].replace(['M', 'm', 'Masculino', 'masculino'], 1)

    return dataframe

## 1.1 Fix data customers

In [5]:
df_clientes = pd.read_csv(raw / 'df_clientes.csv', sep = ',', decimal = '.', header = 0)

df_clientes['fecha_nacimiento'] = pd.to_datetime(df_clientes['fecha_nacimiento'], dayfirst='false', errors = 'coerce')

dataquality.data_quality(df_clientes)

discrete_values(df_clientes, 10)

*****
The dataframe does not contain missing values.
*****
The dataframe does not contain duplicate values.
*****
The dataframe contains values that are not numeric.
Variable not numerit: nombre
Variable not numerit: sexo
Variable not numerit: fecha_nacimiento
*****
The dimension of dataframe is (100, 4)
execution data quality ends.
Discrete variables by value counts: ['sexo']


In [6]:
df_clientes.head()

,id_cliente,nombre,sexo,fecha_nacimiento
0,1001,Maria Casaleggio,F,1991-04-29
1,1002,Sig.ra Rosalia Serlupi,F,2015-08-10
2,1003,Claudia Toninelli-Endrizzi,F,2012-02-09
3,1004,Eugenia Palazzo,F,2008-04-04
4,1005,Sig.ra Lucrezia Pratesi,F,1958-09-22


In [7]:
df_clientes = non_sex(df_clientes, 'sexo')
df_clientes.head()

,id_cliente,nombre,sexo,fecha_nacimiento
0,1001,Maria Casaleggio,0,1991-04-29
1,1002,Sig.ra Rosalia Serlupi,0,2015-08-10
2,1003,Claudia Toninelli-Endrizzi,0,2012-02-09
3,1004,Eugenia Palazzo,0,2008-04-04
4,1005,Sig.ra Lucrezia Pratesi,0,1958-09-22


In [8]:
def calculate_years(dataframe, column, date):
    from datetime import datetime

    print(date)

    if date == 'now':
        current_date = datetime.today()
    else:
        current_date = datetime(date[0], date[1], date[2], 0, 0, 0)
        
    dataframe[column] = (current_date - dataframe[column]).dt.days

    return dataframe

In [9]:
def diff_date(dataframe, column, date):

    ''' 
        This function takes a column and returns the difference with the current date or the entered date.
    '''

    from datetime import datetime

    if date == 'now':
        current_date = datetime.today()
    else:
        current_date = pd.to_datetime(date)
 
    dataframe['age_days'] = (current_date - dataframe[column]).dt.days

    return dataframe

In [10]:
df_clientes = diff_date(df_clientes, "fecha_nacimiento", 'now')
df_clientes.head()

,id_cliente,nombre,sexo,fecha_nacimiento,age_days
0,1001,Maria Casaleggio,0,1991-04-29,11965
1,1002,Sig.ra Rosalia Serlupi,0,2015-08-10,3096
2,1003,Claudia Toninelli-Endrizzi,0,2012-02-09,4374
3,1004,Eugenia Palazzo,0,2008-04-04,5780
4,1005,Sig.ra Lucrezia Pratesi,0,1958-09-22,23872


## 1.1 Fix data products

In [12]:
df_productos = pd.read_csv(raw / 'df_productos.csv', sep = ',', decimal = '.', header = 0)

df_productos.head()

,id_producto,descripcion,articulo,category,precio,production,color,L,tallas
0,759ne,759,Short con tiritas,Short,40.0,55,Negro,ne,1 al 4
1,759bl,759,Short con tiritas,Short,40.0,55,Blanco,bl,1 al 4
2,759az,759,Short con tiritas,Short,40.0,55,Azul,az,1 al 4
3,759ro,759,Short con tiritas,Short,40.0,55,Rojo,ro,1 al 4
4,759ra,759,Short con tiritas,Short,40.0,55,Rosa,ra,1 al 4
